## Import Dependancies

In [62]:
# Dependencies and Setup
import pandas as pd
import numpy as np
#matplot

In [63]:
# File to Load (Remember to Change These)
overview_data = "../Resources/2021VAERSDATA.csv"
symptoms_data = "../Resources/2021VAERSSYMPTOMS.csv"
vaccine_data = "../Resources/2021VAERSVAX.csv"


# Read School and Student Data File and store into Pandas DataFrames
overview_df = pd.read_csv(overview_data)
symptoms_df = pd.read_csv(symptoms_data)
vaccine_df = pd.read_csv(vaccine_data)

C:\Users\tayly\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Clean Data

In [64]:
overview_df.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,...,None,None,NaN,NaN,2,01/01/2021,NaN,Y,NaN,Pcn and bee venom
1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,NaN,NaN,2,01/01/2021,NaN,Y,NaN,"""Dairy"""
2,916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",NaN,...,None,None,NaN,NaN,2,01/01/2021,NaN,NaN,Y,Shellfish
3,916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",NaN,...,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","got measles from measel shot, mums from mumps ...",NaN,2,01/01/2021,NaN,NaN,NaN,"Diclofenac, novacaine, lidocaine, pickles, tom..."
4,916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",NaN,...,Na,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,Na


In [70]:
overview_df.columns

Index(['VAERS_ID', 'AGE_YRS', 'SEX', 'OTHER_MEDS', 'ER_VISIT', 'HOSPITAL',
       'CUR_ILL'],
      dtype='object')

In [71]:
#drop columns from overview_df
overview_df = overview_df[['VAERS_ID', 'AGE_YRS', 'SEX', 'OTHER_MEDS', 'ER_VISIT', 'HOSPITAL', 'CUR_ILL', 'ALLERGIES']]
overview_df.head()

KeyError: "['ALLERGIES'] not in index"

In [53]:
symptoms_df.columns

Index(['VAERS_ID', 'SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5'], dtype='object')

In [54]:
symptoms_df = symptoms_df[['VAERS_ID', 'SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5']]
symptoms_df.head()

,VAERS_ID,SYMPTOM1,SYMPTOM2,SYMPTOM3,SYMPTOM4,SYMPTOM5
0,916600,Dysphagia,Epiglottitis,NaN,NaN,NaN
1,916601,Anxiety,Dyspnoea,NaN,NaN,NaN
2,916602,Chest discomfort,Dysphagia,Pain in extremity,Visual impairment,NaN
3,916603,Dizziness,Fatigue,Mobility decreased,NaN,NaN
4,916604,Injection site erythema,Injection site pruritus,Injection site swelling,Injection site warmth,NaN


In [55]:
vaccine_df.head()

,VAERS_ID,VAX_MANU,VAX_DOSE_SERIES,VAX_SITE
0,916600,MODERNA,1,LA
1,916601,MODERNA,1,RA
2,916602,PFIZER\BIONTECH,1,LA
3,916603,MODERNA,UNK,NaN
4,916604,MODERNA,1,LA


In [56]:
vaccine_df = vaccine_df[['VAERS_ID', 'VAX_MANU', 'VAX_DOSE_SERIES', 'VAX_SITE']]
vaccine_df.head()

,VAERS_ID,VAX_MANU,VAX_DOSE_SERIES,VAX_SITE
0,916600,MODERNA,1,LA
1,916601,MODERNA,1,RA
2,916602,PFIZER\BIONTECH,1,LA
3,916603,MODERNA,UNK,NaN
4,916604,MODERNA,1,LA


In [57]:
# Combine overview with vaccine data  
merge1_df = pd.merge(overview_df, vaccine_df, how="left", on=["VAERS_ID", "VAERS_ID"])

In [58]:
# Combine overview & vaccine with symptoms
merge2_df = pd.merge(merge1_df, symptoms_df, how="left", on=["VAERS_ID", "VAERS_ID"])
merge2_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,ER_VISIT,HOSPITAL,HOSPDAYS,RECOVD,VAX_DATE,NUMDAYS,CUR_ILL,VAX_MANU,VAX_DOSE_SERIES,VAX_SITE,SYMPTOM1,SYMPTOM2,SYMPTOM3,SYMPTOM4,SYMPTOM5
0,916600,33.0,F,NaN,NaN,NaN,NaN,Y,12/28/2020,2.0,None,MODERNA,1,LA,Dysphagia,Epiglottitis,NaN,NaN,NaN
1,916601,73.0,F,NaN,NaN,NaN,NaN,Y,12/31/2020,0.0,Patient residing at nursing facility. See pati...,MODERNA,1,RA,Anxiety,Dyspnoea,NaN,NaN,NaN
2,916602,23.0,F,NaN,NaN,NaN,NaN,U,12/31/2020,0.0,None,PFIZER\BIONTECH,1,LA,Chest discomfort,Dysphagia,Pain in extremity,Visual impairment,NaN
3,916603,58.0,F,NaN,NaN,NaN,NaN,Y,12/23/2020,0.0,kidney infection,MODERNA,UNK,NaN,Dizziness,Fatigue,Mobility decreased,NaN,NaN
4,916604,47.0,F,NaN,NaN,NaN,NaN,N,12/22/2020,7.0,Na,MODERNA,1,LA,Injection site erythema,Injection site pruritus,Injection site swelling,Injection site warmth,NaN


In [59]:
merge2_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,ER_VISIT,HOSPITAL,HOSPDAYS,RECOVD,VAX_DATE,NUMDAYS,CUR_ILL,VAX_MANU,VAX_DOSE_SERIES,VAX_SITE,SYMPTOM1,SYMPTOM2,SYMPTOM3,SYMPTOM4,SYMPTOM5
0,916600,33.0,F,NaN,NaN,NaN,NaN,Y,12/28/2020,2.0,None,MODERNA,1,LA,Dysphagia,Epiglottitis,NaN,NaN,NaN
1,916601,73.0,F,NaN,NaN,NaN,NaN,Y,12/31/2020,0.0,Patient residing at nursing facility. See pati...,MODERNA,1,RA,Anxiety,Dyspnoea,NaN,NaN,NaN
2,916602,23.0,F,NaN,NaN,NaN,NaN,U,12/31/2020,0.0,None,PFIZER\BIONTECH,1,LA,Chest discomfort,Dysphagia,Pain in extremity,Visual impairment,NaN
3,916603,58.0,F,NaN,NaN,NaN,NaN,Y,12/23/2020,0.0,kidney infection,MODERNA,UNK,NaN,Dizziness,Fatigue,Mobility decreased,NaN,NaN
4,916604,47.0,F,NaN,NaN,NaN,NaN,N,12/22/2020,7.0,Na,MODERNA,1,LA,Injection site erythema,Injection site pruritus,Injection site swelling,Injection site warmth,NaN


In [67]:
merge2_df.to_csv('training_data.csv')